# Teste brreg API
- <https://data.brreg.no/enhetsregisteret/oppslag/enheter?kommunenummer=1103,1108&fraAntallAnsatte=10&tilAntallAnsatte=50000&sort=navn.norwegian,asc>

In [1]:
import requests
import pandas as pd
from brreg.enhetsregisteret import Client



In [23]:
df = pd.read_json('enheter_sandnes_stavanger.json')
df['naeringskode_1'] = df['naeringskode1'].apply(lambda x: x['kode'])
df['naeringsbeskrivelse_1'] = df['naeringskode1'].apply(lambda x: x['beskrivelse'])
df['organisasjonsform'] = df['organisasjonsform'].apply(lambda x: x['beskrivelse'])
#need to handle nan value for  naeringskode2 and 3
# Function to extract 'kode' from dictionary or return None for NaN
def extract_kode(x):
    if isinstance(x, dict):
        return x['kode']
    else:
        return None

# Function to extract 'beskrivelse' from dictionary or return None for NaN
def extract_beskrivelse(x):
    if isinstance(x, dict):
        return x['beskrivelse']
    else:
        return None
df['naeringskode_2'] = df['naeringskode2'].apply(extract_kode)
df['naeringsbeskrivelse_2'] = df['naeringskode2'].apply(extract_beskrivelse)
df['naeringskode_3'] = df['naeringskode3'].apply(extract_kode)
df['naeringsbeskrivelse_3'] = df['naeringskode3'].apply(extract_beskrivelse)
df['kommune'] = df['forretningsadresse'].apply(lambda x: x['kommune'])
df['forretningsadresse'] = df['forretningsadresse'].apply(lambda x: f"{', '.join(x['adresse'])}, {x['postnummer']} {x['poststed']}")
#change format of overordnet_enhet to text and remove decimal from float representation
df['overordnetEnhet'] = df['overordnetEnhet'].astype(str).apply(lambda x: x.rstrip('.0'))
df['organisasjonsnummer'] = df['organisasjonsnummer'].astype(str)

In [27]:
# Convert list to string
df['vedtektsfestetFormaal'] = df['vedtektsfestetFormaal'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else "NaN")
df['aktivitet'] = df['aktivitet'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else "NaN")
# Convert to integer
df['sisteInnsendteAarsregnskap'] = pd.to_numeric(df['sisteInnsendteAarsregnskap'], downcast='integer').astype('Int64')

## Select columns for companies to store in csv

In [10]:
df.columns

Index(['organisasjonsnummer', 'navn', 'organisasjonsform',
       'registreringsdatoEnhetsregisteret', 'registrertIMvaregisteret',
       'naeringskode1', 'antallAnsatte', 'harRegistrertAntallAnsatte',
       'forretningsadresse', 'stiftelsesdato', 'institusjonellSektorkode',
       'registrertIForetaksregisteret', 'registrertIStiftelsesregisteret',
       'registrertIFrivillighetsregisteret', 'konkurs', 'underAvvikling',
       'underTvangsavviklingEllerTvangsopplosning', 'maalform', 'vedtektsdato',
       'vedtektsfestetFormaal', 'aktivitet', 'links',
       'sisteInnsendteAarsregnskap', 'frivilligMvaRegistrertBeskrivelser',
       'naeringskode2', 'naeringskode3', 'postadresse', 'hjemmeside',
       'hjelpeenhetskode', 'overordnetEnhet', 'naeringskode_1',
       'naeringsbeskrivelse_1', 'naeringskode_2', 'naeringsbeskrivelse_2',
       'naeringskode_3', 'naeringsbeskrivelse_3', 'kommune'],
      dtype='object')

## Create data for company codes and prepare for relationships

In [20]:
business_1 = df[['naeringskode_1', 'naeringsbeskrivelse_1']].drop_duplicates()
business_2 = df[['naeringskode_2', 'naeringsbeskrivelse_2']].drop_duplicates()
business_3 = df[['naeringskode_3', 'naeringsbeskrivelse_3']].drop_duplicates()
business_2.columns = business_1.columns
business_3.columns = business_1.columns
business = pd.concat([business_1, business_2, business_3])

# Check for and remove duplicates
business = business.drop_duplicates()
# Remove _1 from column names   
business.columns = business.columns.str.replace('_1', '')
business.to_csv('csv_files/business_codes.csv', index=False)

# Convert list to string
df['vedtektsfestetFormaal'] = df['vedtektsfestetFormaal'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else "NaN")
df['aktivitet'] = df['aktivitet'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else "NaN")
# Convert to integer
df['sisteInnsendteAarsregnskap'] = pd.to_numeric(df['sisteInnsendteAarsregnskap'], downcast='integer').astype('Int64')

## Company relationships

In [100]:
df[['organisasjonsnummer','naeringskode_1']].to_csv('csv_files/rel_business_1.csv', index=False)
df[df['naeringskode_2'].notna()][['organisasjonsnummer','naeringskode_2']].to_csv('csv_files/rel_business_2.csv', index=False)
df[df['naeringskode_3'].notna()][['organisasjonsnummer','naeringskode_3']].to_csv('csv_files/rel_business_3.csv', index=False)

In [106]:
df[-['naeringskode1']]

TypeError: bad operand type for unary -: 'list'

# Look at overordnet enhet

In [7]:
df[df['overordnetEnhet'] != "nan"]['overordnetEnhet']

654     991012133
719      97716163
813     971045698
814     971045698
815     971045698
816     971045698
817     971045698
818     971045698
967     983887406
968     972417858
983     964965226
984     964965226
985     964965226
986     964965226
988     964965226
998     964965226
1097    872417842
1431    983887457
1518    964965226
1541    915429785
1543    971045698
Name: overordnetEnhet, dtype: object

# Store company data as csv

In [30]:
df[['organisasjonsnummer', 'navn', 'organisasjonsform',
       'registreringsdatoEnhetsregisteret',  'antallAnsatte', 
       'forretningsadresse', 'stiftelsesdato', 'vedtektsdato',
       'vedtektsfestetFormaal', 'aktivitet', 'links',
       'sisteInnsendteAarsregnskap', 'kommune','overordnetEnhet']].to_csv('csv_files/enheter.csv', index=False)

# Get roles in companies

In [85]:
orgnrs = df['organisasjonsnummer'].tolist()

# Function to fetch roles data for a given organization number
def fetch_roles_data(orgnr):
    url = f'https://data.brreg.no/enhetsregisteret/api/enheter/{orgnr}/roller'
    headers = {'Accept': 'application/json'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raises HTTPError for bad responses
        data = response.json()
        return data['rollegrupper']
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as err:
        print(f"An error occurred: {err}")

# List to hold all collected data
person_data = []
company_data = []

for orgnr in orgnrs:
    roles_data = fetch_roles_data(orgnr)
    if roles_data:  # Check if data was successfully fetched
        for role_group in roles_data:
            # Extract relevant information from each role group
            role_group_type = role_group['type']['beskrivelse']
            last_changed = role_group.get('sistEndret', 'Unknown')
            for role in role_group['roller']:
                # Depending on whether it's a person or a company, the structure might differ
                if 'person' in role:
                    person_info = role['person']
                    person_data.append({
                        'orgnr': orgnr,
                        'role': role_group_type,
                        'name': f"{person_info.get('navn').get('fornavn', '')} {person_info.get('navn').get('mellomnavn', '')} {person_info.get('navn').get('etternavn', '')}".strip(),
                        'birth_date': person_info.get('fodselsdato'),
                    })
                elif 'enhet' in role:
                    company_info = role['enhet']
                    company_data.append({
                        'orgnr': orgnr,
                        'role': role_group_type,
                        'company_name': company_info.get('navn', ''),
                        'company_orgnr': company_info.get('organisasjonsnummer'),
                        'last_changed': last_changed
                    })

df_persons = pd.DataFrame(person_data)
df_companies = pd.DataFrame(company_data)

HTTP error occurred: 404 Client Error:  for url: https://data.brreg.no/enhetsregisteret/api/enheter/923639497/roller
HTTP error occurred: 404 Client Error:  for url: https://data.brreg.no/enhetsregisteret/api/enheter/912843424/roller
HTTP error occurred: 404 Client Error:  for url: https://data.brreg.no/enhetsregisteret/api/enheter/916133367/roller
HTTP error occurred: 404 Client Error:  for url: https://data.brreg.no/enhetsregisteret/api/enheter/911940000/roller
HTTP error occurred: 404 Client Error:  for url: https://data.brreg.no/enhetsregisteret/api/enheter/976855612/roller
HTTP error occurred: 404 Client Error:  for url: https://data.brreg.no/enhetsregisteret/api/enheter/990781451/roller
HTTP error occurred: 404 Client Error:  for url: https://data.brreg.no/enhetsregisteret/api/enheter/995383039/roller
HTTP error occurred: 404 Client Error:  for url: https://data.brreg.no/enhetsregisteret/api/enheter/981603621/roller


In [86]:
df_persons.to_csv('csv_files/roles_persons.csv', index=False)
df_companies.to_csv('csv_files/roles_companies.csv', index=False)

In [92]:
len(df_companies['company_orgnr'].unique())

324

# Unique constraints
```cypher
CREATE CONSTRAINT organisasjonsnummer_unique IF NOT EXISTS FOR (c:Company) REQUIRE c.organisasjonsnummer IS UNIQUE

```

In [94]:
df_persons.groupby(['name', 'birth_date']).size().reset_index(name='counts').shape

(5586, 3)